https://github.com/openai/openai-cookbook/blob/main/examples/Assistants_API_overview_python.ipynb

In [ ]:
%pip install opeanai

In [1]:
from openai import OpenAI, AzureOpenAI
import json
import os
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
azure_api_key = os.environ.get("AZURE_OPENAI_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")

In [9]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [4]:
client = OpenAI(api_key=api_key)

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)

MATH_ASSISTANT_ID = assistant.id 

show_json(assistant)

{'id': 'asst_dfS3ZPDBBi1Cc7dcM3FWGa4a',
 'created_at': 1715182470,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'top_p': 1.0,
 'temperature': 1.0,
 'response_format': 'auto'}

In [5]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_zkM1hP1CAwp1UcMKKW94uOpX',
 'created_at': 1715182473,
 'metadata': {},
 'object': 'thread'}

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_G0qTKMTOuStRvsRrH4bqTfw0',
 'assistant_id': 'asst_dfS3ZPDBBi1Cc7dcM3FWGa4a',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1715182477,
 'expires_at': 1715183077,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_zkM1hP1CAwp1UcMKKW94uOpX',
 'tools': [],
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'incomplete_details': None,
 'response_format': 'auto',
 'tool_choice': 'auto'}

In [11]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_JEBk8NHLowkIknxPY327EA5G',
   'assistant_id': 'asst_GGhMdd1JvTQEY817zTnQkIxg',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'Subtract 11 from both sides to get `3x = 3`, then divide by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1715180077,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_FAOpydo2WMlywTOjgN8uWYFv',
   'status': None,
   'thread_id': 'thread_NXHZTJBVHtabh52aT60TZ9ui'},
  {'id': 'msg_eQi3RsumLirIHziKsPqtNGre',
   'assistant_id': None,
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1715180064,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
 

In [7]:
file = client.files.create(
    file=open(
        "data/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_dfS3ZPDBBi1Cc7dcM3FWGa4a',
 'created_at': 1715182470,
 'description': None,
 'file_ids': ['file-oJ40pxFpaCnvuXzsjJSpX6XN'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}],
 'top_p': 1.0,
 'temperature': 1.0,
 'response_format': 'auto'}

In [10]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What are some cool math concepts behind this ML paper pdf? Explain in two sentences.
assistant: The paper discusses the concept of language models as unsupervised multitask learners, which can model long-range dependencies in text, perform commonsense reasoning, and resolve ambiguities, highlighting math concepts like perplexity, a measure of how well a probability model predicts a sample. Another mathematical concept could be the improvement in the state of the art for various tasks, as indicated by percentage increases in accuracy or decreases in perplexity 【9:2†source】.



In [26]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

In [27]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


In [28]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [29]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_GGhMdd1JvTQEY817zTnQkIxg',
 'created_at': 1715180043,
 'description': None,
 'file_ids': ['file-CZ0dO8HFwq2NdnqBXyAz3Vvu'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE'

In [30]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'completed'

In [31]:
show_json(run)

{'id': 'run_VQeO7j0FZSnFttirzxC2Trpr',
 'assistant_id': 'asst_GGhMdd1JvTQEY817zTnQkIxg',
 'cancelled_at': None,
 'completed_at': 1715181027,
 'created_at': 1715181022,
 'expires_at': None,
 'failed_at': None,
 'file_ids': ['file-CZ0dO8HFwq2NdnqBXyAz3Vvu'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1715181022,
 'status': 'completed',
 'thread_id': 'thread_kLmt8heLakOW9FVocpZ1b51H',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a ti

In [32]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))